## ENGRI 1120 Mole Balance Flux Balance Analysis Example

### Introduction

### Example setup

In [1]:
import Pkg; Pkg.activate("."); Pkg.resolve(); Pkg.instantiate();

  Activating project at `~/Desktop/julia_work/ENGRI-1120-IntroToChemE-Example-Notebooks/notebooks-jupyter/ENGRI-1120-Toy-FBA-Example`
  No Changes to `~/Desktop/julia_work/ENGRI-1120-IntroToChemE-Example-Notebooks/notebooks-jupyter/ENGRI-1120-Toy-FBA-Example/Project.toml`
  No Changes to `~/Desktop/julia_work/ENGRI-1120-IntroToChemE-Example-Notebooks/notebooks-jupyter/ENGRI-1120-Toy-FBA-Example/Manifest.toml`


In [2]:
# load reqd packages and set paths -
using PrettyTables
using GLPK

# setup paths -
const _ROOT = pwd();

In [3]:
include("ENGRI-1120-Example-CodeLib.jl");

#### a) Build the stoichiometric matrix $S$

<img src="figs/Fig-expa-ToyNetwork.png" style="width:50%">

In [4]:
# Setup a collection of reaction strings -
reaction_array = Array{String,1}()

# encode the reactions -
# internal reactions -
push!(reaction_array,"v₁,A,B,false")
push!(reaction_array,"v₂,B,C,true")
push!(reaction_array,"v₃,C,D,true")
push!(reaction_array,"v₄,C,E,false")

# exchange reactions -
push!(reaction_array,"b₁,Ax,A,true")
push!(reaction_array,"b₂,Bx,B,true")
push!(reaction_array,"b₃,D,Dx,true")
push!(reaction_array,"b₄,E,Ex,true")

# compute the stoichiometric matrix -
# the optional expand arguement = should we split reversible reactions? (default: false)
(S, species_array, reaction_name_array) = build_stoichiometric_matrix(reaction_array; 
    expand=false);

In [5]:
(ℳ, ℛ) = size(S);

In [6]:
[1:ℳ species_array]

9×2 Matrix{Any}:
 1  "A"
 2  "Ax"
 3  "B"
 4  "Bx"
 5  "C"
 6  "D"
 7  "Dx"
 8  "E"
 9  "Ex"

In [7]:
[1:ℛ reaction_name_array]

8×2 Matrix{Any}:
 1  "v₁"
 2  "v₂"
 3  "v₃"
 4  "v₄"
 5  "b₁"
 6  "b₂"
 7  "b₃"
 8  "b₄"

#### b) Build the reaction bounds array

In [8]:
# setup the bounds array -
flux_bounds_array = zeros(ℛ,2);
flux_bounds_array[:,2] .= 100.0; # set a default value for the *upper* bound on the flux

# set an upper bound on v₂ -
flux_bounds_array[2,2] = 1.0;

#### c) Build the species bounds array

In [9]:
# setup the species bounds array -

# we know from out theory, that that the lower bound is -1*sum of the inputs 
ṅ₁ = zeros(ℳ);
ṅ₂ = zeros(ℳ);

# suppose we supply Ax in stream 1, and Bx in stream 2
ṅ₁[2] = 20.0; # supply Ax -
ṅ₂[4] = 5.0;  # supply Bx -

# setup -
species_bounds_array = [-1*(ṅ₁ .+ ṅ₂) 1000.0*ones(ℳ)];

#### d) Set the objective coefficient vector

In [10]:
# setup the objective vector -
c = zeros(ℛ);
c[7] = -1.0;

#### e) Estimate the extent through the network

In [11]:
results = compute_optimal_extent(S, flux_bounds_array, species_bounds_array, c);

In [12]:
# check:
println("Exit flag: $(results.exit_flag) and status flag: $(results.status_flag)")

Exit flag: 0 and status flag: 5


In [13]:
# get the reaction extent vector -
ϵ̇ = results.calculated_flux_array

# build a table -
optimal_extent_table_data = Array{Any,2}(undef, ℛ, 2);
for i ∈ 1:ℛ
    optimal_extent_table_data[i,1] = reaction_name_array[i]
    optimal_extent_table_data[i,2] = ϵ̇[i]
end

# build header -
optimal_table_header = (["Reaction", "ϵ̇ᵢ"], ["", "mol/time"])

# show table -
pretty_table(optimal_extent_table_data; header=optimal_table_header)

┌──────────┬──────────┐
│ Reaction │       ϵ̇ᵢ │
│          │ mol/time │
├──────────┼──────────┤
│       v₁ │      0.0 │
│       v₂ │      1.0 │
│       v₃ │      1.0 │
│       v₄ │      0.0 │
│       b₁ │      0.0 │
│       b₂ │      1.0 │
│       b₃ │      1.0 │
│       b₄ │      0.0 │
└──────────┴──────────┘


In [14]:
# compute the output compostion -
ṅ₃ = ṅ₂ + ṅ₁ + S*ϵ̇;

# compute the change because of the reaction -
Δ = S*ϵ̇;


# build flow table -
flow_table_data = Array{Any,2}(undef, ℳ, 5);
for i ∈ 1:ℳ
    flow_table_data[i,1] = species_array[i]
    flow_table_data[i,2] = ṅ₁[i]
    flow_table_data[i,3] = ṅ₂[i]
    flow_table_data[i,4] = ṅ₃[i]
    flow_table_data[i,5] = Δ[i]
end

# setup header -
flow_header_data = (["Species", "ṅ₁", "ṅ₂","ṅ₃", "Δ"], ["", "mol/time", "mol/time", "mol/time", "mol/time"]) 

# show -
pretty_table(flow_table_data; header = flow_header_data)

┌─────────┬───────┬───────┬───────┬───────┐
│ Species │    ṅ₁ │    ṅ₂ │    ṅ₃ │     Δ │
│         │ mol/t │ mol/t │ mol/t │ mol/t │
├─────────┼───────┼───────┼───────┼───────┤
│       A │   0.0 │   0.0 │   0.0 │   0.0 │
│      Ax │  20.0 │   0.0 │  20.0 │   0.0 │
│       B │   0.0 │   0.0 │   0.0 │   0.0 │
│      Bx │   0.0 │   5.0 │   4.0 │  -1.0 │
│       C │   0.0 │   0.0 │   0.0 │   0.0 │
│       D │   0.0 │   0.0 │   0.0 │   0.0 │
│      Dx │   0.0 │   0.0 │   1.0 │   1.0 │
│       E │   0.0 │   0.0 │   0.0 │   0.0 │
│      Ex │   0.0 │   0.0 │   0.0 │   0.0 │
└─────────┴───────┴───────┴───────┴───────┘
